In [19]:
import requests
from bs4 import BeautifulSoup
import json
import re
from nltk import word_tokenize
import pandas as pd
from urllib.request import urlopen
import requests

### Import list of names from json file:

In [20]:
CharacterNames = []
with open('characters.json') as json_file:
    data = json.load(json_file)
    for p in data['characters']:
        CharacterNames.append(p['characterName'])


In [21]:
s1 = ['/wiki/Winter_Is_Coming', '/wiki/The_Kingsroad', '/wiki/Lord_Snow', 
      '/wiki/Cripples,_Bastards,_and_Broken_Things','/wiki/The_Wolf_and_the_Lion', '/wiki/A_Golden_Crown',
      '/wiki/You_Win_or_You_Die', '/wiki/The_Pointy_End', '/wiki/Baelor',  '/wiki/Fire_and_Blood']


s2 = ['/wiki/What_Is_Dead_May_Never_Die', '/wiki/The_Night_Lands', '/wiki/The_North_Remembers', 
     '/wiki/A_Man_Without_Honor', '/wiki/The_Old_Gods_and_the_New', '/wiki/The_Ghost_of_Harrenhal',
      '/wiki/Garden_of_Bones_(episode)', '/wiki/Valar_Morghulis', '/wiki/Blackwater', '/wiki/The_Prince_of_Winterfell']

s3 = ['/wiki/Valar_Dohaeris', '/wiki/Dark_Wings,_Dark_Words', '/wiki/Walk_of_Punishment_(episode)', 
     '/wiki/And_Now_His_Watch_Is_Ended','/wiki/Kissed_by_Fire', '/wiki/The_Climb', 
      '/wiki/The_Bear_and_the_Maiden_Fair_(episode)',
     '/wiki/Second_Sons_(episode)', '/wiki/The_Rains_of_Castamere_(episode)', '/wiki/Mhysa']

s4 = ['/wiki/Oathkeeper_(episode)', '/wiki/Breaker_of_Chains', '/wiki/The_Lion_and_the_Rose',
      '/wiki/Two_Swords', '/wiki/Mockingbird','/wiki/The_Laws_of_Gods_and_Men','/wiki/First_of_His_Name',
     '/wiki/The_Children','/wiki/The_Watchers_on_the_Wall','/wiki/The_Mountain_and_the_Viper']

s5 = ['/wiki/Sons_of_the_Harpy_(episode)', '/wiki/High_Sparrow_(episode)',
      '/wiki/The_House_of_Black_and_White', '/wiki/The_Wars_To_Come', 
     '/wiki/Mother%27s_Mercy', '/wiki/The_Dance_of_Dragons','/wiki/Hardhome_(episode)', 
      '/wiki/The_Gift', '/wiki/Unbowed,_Unbent,_Unbroken', '/wiki/Kill_the_Boy']


### Get relevant paths

In [22]:
def GetUrls(url):

    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    
    AllLinks = []
    for link in soup.find_all('a'):
        AllLinks.append(link.get('href'))
    
    idx = AllLinks.index('/wiki/Season_8')
    
    AllLinks = AllLinks[idx:idx+26]
    
    indices = [AllLinks.index("/wiki/Season_" +str(i)) for i in [8, 7, 6]]
    
    SeaLen = [indices[i+1]-indices[i]-1 for i in range(2)] + [len(AllLinks)-indices[2]-1]
    Seasons = SeaLen[0]*['Season_8'] + SeaLen[1]*['Season_7'] + SeaLen[2]*['Season_6']

    j = 0
    for i in indices: 
        del(AllLinks[i-j])
        j+=1

    return AllLinks, Seasons

def SeasonList(paths, season):
    
    return season*len(paths)
    
    

### Get documents

In [23]:
def GetDocuments(root, endpath):
    url = root + endpath
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        # there may be more elements you don't want, such as "style", etc.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
            
    return soup, text, output

### Accesss all captions

In [24]:
def GetSectionName(soup):

    AllH = soup.find_all(['h2', 'h3'])
    for header in AllH:
        if header.span is not None: 
            header.span.unwrap()

    AllHeaders = [header.text for header in AllH]
    if 'Summary' in AllHeaders:
        Pre = AllHeaders.index('Summary')
    elif 'Synopsis' in AllHeaders: 
        Pre = AllHeaders.index('Synopsis')
    else: 
        print("something went wrong")
        return 
        
    Post = AllHeaders.index('Appearances')

    HeadlineName = AllHeaders[Pre+1:Post]
    return HeadlineName

### Access all releveant text (including captions)

In [25]:
def GetText(HeadlineNames):
    HeadlineNames += ['Appearances']

    # Unwrap text
    Texts = soup.find_all(['p','h2', 'h3'])
    for section in Texts: 
        if section is not None:
            if section.a is not None: 
                section.a.unwrap()
            elif section.span is not None: 
                section.span.unwrap()

    #S tore text in list
    Texts = [t.text for t in Texts]

    # Find section indices in Texts
    CaptionIndices = []
    for caption in HeadlineNames:         
        CaptionIndices.append(Texts.index(caption))

    # Store texts according to section
    SectionText = []
    for i in range(len(CaptionIndices)-1):
        SectionText.append(Texts[CaptionIndices[i]+1:CaptionIndices[i+1]])


    return SectionText



In [26]:
#Initialise dataframe
df = pd.DataFrame(columns = ["season", "caption", "text"])

# Get all urls
url = "https://gameofthrones.fandom.com/wiki/"
EndAllUrls, Seasons = GetUrls(url)

#Add season 1 through 5
EndAllUrls = s1 +s2 + s3 + s4 + s5 + EndAllUrls 
s = []
for i,j in enumerate([s1, s2, s3, s4, s5]):
    s += SeasonList(j,['season' + str(i+1)])
Seasons = s + Seasons


root = 'https://gameofthrones.fandom.com'

AllCaptions = []
AllTexts = []
AllSeasons = []
for endpath, season in zip(EndAllUrls, Seasons):
    
    # Webscrabe
    print(endpath)
    soup, text, output = GetDocuments(root, endpath)

    # Get section names
    Captions = GetSectionName(soup)

    # Get texts
    SectionText = GetText(Captions)
    
    AllCaptions = AllCaptions + Captions[:-1]
    AllTexts = AllTexts + SectionText
    AllSeasons = AllSeasons + [season] * len(Captions[:-1])
    
    
# Store in dataframe
df['caption'] = AllCaptions
df['text'] = AllTexts
df['season'] = AllSeasons


/wiki/Winter_Is_Coming
/wiki/The_Kingsroad
/wiki/Lord_Snow
/wiki/Cripples,_Bastards,_and_Broken_Things
/wiki/The_Wolf_and_the_Lion
/wiki/A_Golden_Crown
/wiki/You_Win_or_You_Die
/wiki/The_Pointy_End
/wiki/Baelor
/wiki/Fire_and_Blood
/wiki/What_Is_Dead_May_Never_Die
/wiki/The_Night_Lands
/wiki/The_North_Remembers
/wiki/A_Man_Without_Honor
/wiki/The_Old_Gods_and_the_New
/wiki/The_Ghost_of_Harrenhal
/wiki/Garden_of_Bones_(episode)
/wiki/Valar_Morghulis
/wiki/Blackwater
/wiki/The_Prince_of_Winterfell
/wiki/Valar_Dohaeris
/wiki/Dark_Wings,_Dark_Words
/wiki/Walk_of_Punishment_(episode)
/wiki/And_Now_His_Watch_Is_Ended
/wiki/Kissed_by_Fire
/wiki/The_Climb
/wiki/The_Bear_and_the_Maiden_Fair_(episode)
/wiki/Second_Sons_(episode)
/wiki/The_Rains_of_Castamere_(episode)
/wiki/Mhysa
/wiki/Oathkeeper_(episode)
/wiki/Breaker_of_Chains
/wiki/The_Lion_and_the_Rose
/wiki/Two_Swords
/wiki/Mockingbird
/wiki/The_Laws_of_Gods_and_Men
/wiki/First_of_His_Name
/wiki/The_Children
/wiki/The_Watchers_on_the_Wall
/

In [27]:
df.to_csv('GOT.csv')

In [28]:
df

,season,caption,text
0,season1,Beyond the Wall,[Three rangers of the Night's Watch: Ser Wayma...
1,season1,In Pentos,"[In the Free City of Pentos, across the Narrow..."
2,season1,In King's Landing,"[In King's Landing, the capital city of the Se..."
3,season1,In the North,"[Ned Stark draws his sword., Some time later, ..."
4,season1,In the Dothraki Sea,[The Dothraki khalasar led by Khal Drogo have ...
...,...,...,...
391,Season_6,In Oldtown,"[Sam in the Citadel's library., Sam, Gilly, an..."
392,Season_6,Beyond the Wall,[Bran sees the vision of his father and Aunt L...
393,Season_6,At Winterfell,[Melisandre and Jon Snow are in Winterfell's m...
394,Season_6,In Dorne,[Ellaria and the Sand Snakes meet with Olenna ...
